# Fancy but Useful Tensor Operations

In [2]:
import torch
import numpy as np

## 基础语法

### 创建操作

#### 和 NumPy 数组的转换

In [8]:
# 使用 `torch.from_numpy()` (共享内存)
a_array = np.array([1, 2, 3])
a_tensor = torch.from_numpy(a_array)
print("Original PyTorch tensor:", a_tensor)
a_array[0] = 10  # 修改 NumPy 数组
print("Tensor after modifying NumPy array:", a_tensor)

Original PyTorch tensor: tensor([1, 2, 3])
Tensor after modifying NumPy array: tensor([10,  2,  3])


In [10]:
# 使用 `torch.tensor()` (复制内存)
b_array = np.array([[1, 2], [3, 4]])
b_tensor = torch.tensor(b_array)
print("Original PyTorch tensor:")
print(b_tensor)
b_array[0, 0] = 10  # 修改 NumPy 数组
print("Tensor after modifying NumPy array:")
print(b_tensor)

Original PyTorch tensor:
tensor([[1, 2],
        [3, 4]])
Tensor after modifying NumPy array:
tensor([[1, 2],
        [3, 4]])


In [11]:
# 从 Tensor 转换为 NumPy 数组
assert torch.cuda.is_available(), "CUDA is not available"
c_tensor = torch.tensor([1, 2, 3], device='cuda')  # 创建一个在 GPU 上的 Tensor
try: # GPU 上的 Tensor 不能直接转换为 NumPy 数组
    c_array = c_tensor.numpy()
except Exception as e:
    print(e)
c_array = c_tensor.cpu().numpy()
print("NumPy array from CPU tensor:", c_array)

can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.
NumPy array from CPU tensor: [1 2 3]


#### 特殊 Tensor 构造

In [6]:
# 构造全 0 的 Tensor
a = torch.zeros(1, 2)
print("Tensor of zeros:")
print(a)

# 构造全 1 的 Tensor
b = torch.ones(2, 3)
print("\nTensor of ones:")
print(b)

# 构造随机 Tensor
c = torch.rand(3, 4)
print("\nRandom Tensor:")
print(c)

Tensor of zeros:
tensor([[0., 0.]])

Tensor of ones:
tensor([[1., 1., 1.],
        [1., 1., 1.]])

Random Tensor:
tensor([[0.0978, 0.7573, 0.7558, 0.5040],
        [0.0750, 0.7821, 0.8679, 0.0630],
        [0.6995, 0.3050, 0.4670, 0.4738]])


### 基本属性

In [12]:
assert torch.cuda.is_available(), "CUDA is not available"
a = torch.tensor([[[1, 2, 3], [4, 5, 6]]], device='cuda')
print("Rank of a:", a.dim())
print("Shape of a:", a.shape)
print("Shape of dim 1:", a.shape[1])
print("Device of a:", a.device)

Rank of a: 3
Shape of a: torch.Size([1, 2, 3])
Shape of dim 1: 2
Device of a: cuda:0
